## What is ARIMA?
ARIMA is a class of statistical modelling for analyzing and forecasting time series data.
It is an acronym that stands for **A**uto**R**egressive **I**ntegrated **M**oving Average. It is a generalization of the simpler AutoRegressive Moving Average and adds the notion of integration. It's used in Statistics, Econometrics, and Time Series Analysis.
- 'Autoregression' is model that uses the dependent relationship between an observation and some number of lagged observations (fixed amount of passing time).
- 'Integrated' is use of differencing of raw observations (i.e. subtracting an observation from an observation at the previous time step) in order to make the time series stationary.
- 'Moving Average' is the model that uses the dependency between an observation and residual errors from a moving average model applied to lagged observations.

Non-seasonal ARIMA models are generally denoted as "ARIMA(p,d,q)" where p,d,q are non-negative integers. 'p' is the order (the number of time lags), 'd' is the number of times the data have had past values subtracted (also known as the degree of differencing), and 'q' is the order of the moving average. Seasonal ARIMA models are usually denoted ARIMA(p,d,q)(P,D,Q)m, where m refers to the number of periods in each season, and the uppercase P,D,Q refer to the autoregressive, differencing, and moving average terms for the seasonal part of the ARIMA model.

In [ ]:
# load and plot dataset
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages, helps with deprecation

# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo-sales.csv' , header=0, index_col=0, parse_dates=True,
                  squeeze=True, date_parser=parser)

# summarize first few rows
print(series.head())

# line plot
series.plot()
pyplot.show()

In [ ]:
# autocorrelation plot of time series
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages, helps with deprecation

# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo-sales.csv', header=0, index_col=0, parse_dates=True,
                  squeeze=True, date_parser=parser)

# autocorrelation plot
autocorrelation_plot(series)
pyplot.show()

## Using the Statsmodels library to create an ARIMA model in Python
The entire Shampoo Sales dataset is fit to an ARIMA model and is reviewed for residual errors. The model is first set to (5,1,0) which sets the lag value to 5 for autoregression, uses a difference order of 1 to make the time series stationary, and uses a moving average model of 0. Excess debug information is turned off by setting the disp
argument to 0.

A summary is printed followed by line and density plots. Finally, any residual errors are displayed to see if there is any bias in the data.

In [ ]:
# fit an ARIMA model and plot residual errors
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages, helps with deprecation

# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo-sales.csv', header=0, index_col=0, parse_dates=True,
                  squeeze=True, date_parser=parser)

# fit model
model = ARIMA(series, order=(5,1,0))
model_fit = model.fit(disp=0)

# summary of fit model
print(model_fit.summary())

# line plot of residuals
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()

# density plot of residuals
residuals.plot(kind= 'kde' )
pyplot.show()

# summary stats of residuals
print(residuals.describe())

## Rolling Forcast ARIMA Model

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages, helps with deprecation

# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo-sales.csv', header=0, index_col=0, parse_dates=True,
                  squeeze=True, date_parser=parser)

# split into train and test sets
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# walk-forward validation
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# plot forecasts against actual outcomes
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()